In [3]:
from datasets import load_dataset

ds = load_dataset("birdsql/bird_sql_dev_20251106")

In [4]:
import sqlite3
import pandas as pd
sql3 = sqlite3.connect('/data/workspace/sogang/sqlagent/california_schools/california_schools.sqlite')
df = ds['dev_20251106'].to_pandas()

In [9]:
cursor = sql3.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables:", tables)

for table in tables:
    table_name = table[0]
    cursor.execute(f"PRAGMA table_info({table_name});")
    schema = cursor.fetchall()
    print(f"\nSchema for {table_name}:")
    for col in schema:
        print(col)

Tables: [('frpm',), ('satscores',), ('schools',)]

Schema for frpm:
(0, 'CDSCode', 'TEXT', 1, None, 1)
(1, 'Academic Year', 'TEXT', 0, None, 0)
(2, 'County Code', 'TEXT', 0, None, 0)
(3, 'District Code', 'INTEGER', 0, None, 0)
(4, 'School Code', 'TEXT', 0, None, 0)
(5, 'County Name', 'TEXT', 0, None, 0)
(6, 'District Name', 'TEXT', 0, None, 0)
(7, 'School Name', 'TEXT', 0, None, 0)
(8, 'District Type', 'TEXT', 0, None, 0)
(9, 'School Type', 'TEXT', 0, None, 0)
(10, 'Educational Option Type', 'TEXT', 0, None, 0)
(11, 'NSLP Provision Status', 'TEXT', 0, None, 0)
(12, 'Charter School (Y/N)', 'INTEGER', 0, None, 0)
(13, 'Charter School Number', 'TEXT', 0, None, 0)
(14, 'Charter Funding Type', 'TEXT', 0, None, 0)
(15, 'IRC', 'INTEGER', 0, None, 0)
(16, 'Low Grade', 'TEXT', 0, None, 0)
(17, 'High Grade', 'TEXT', 0, None, 0)
(18, 'Enrollment (K-12)', 'REAL', 0, None, 0)
(19, 'Free Meal Count (K-12)', 'REAL', 0, None, 0)
(20, 'Percent (%) Eligible Free (K-12)', 'REAL', 0, None, 0)
(21, 'FRPM C

In [2]:
df = ds['dev_20251106'].to_pandas()

In [3]:
hard_df = df[df['difficulty'] == 'challenging']


In [ ]:
prompt_updater_prompt = """당신은 대규모 언어 모델을 위한 효과적인 시스템 프롬프트를 만드는 프롬프트 엔지니어링 전문가입니다.
당신의 임무는 사용자 피드백과 LLM의 생성 결과를 바탕으로 주어진 시스템 프롬프트를 개선하는 것입니다.

현재 시스템 프롬프트는 다음과 같습니다:
--------------------
{current_prompt}
--------------------

위 프롬프트를 사용하여 생성된 LLM 결과에 대한 사용자의 피드백은 다음과 같습니다:
--------------------
{feedback_prompt}
--------------------

다음은 LLM이 실제로 생성한 결과입니다:
--------------------
{llm_result}
--------------------

피드백과 LLM 결과를 분석하여 현재 시스템 프롬프트의 문제점을 파악하고, 이를 해결하기 위해 시스템 프롬프트를 다시 작성해주세요.

새로운 시스템 프롬프트는 다음 조건을 만족해야 합니다:
- LLM이 이해하기 쉽고 명확해야 합니다.
- 원하는 결과물을 생성하도록 LLM을 구체적으로 안내해야 합니다.
- 다양한 유사 입력에 대응할 수 있도록 충분히 일반적이어야 합니다.
- 사용자의 피드백을 반영해야 합니다.

출력은 오직 수정된 시스템 프롬프트여야 합니다. 당신의 의견이나 설명, 서문 등은 포함하지 마세요."""
evaluation_prompt = """당신은 데이터베이스 쿼리 결과의 정확성을 평가하는 QA 전문가입니다.
당신의 임무는 'evidence' (정답) 데이터와 'llm_result' (LLM이 생성한 쿼리의 실행 결과)를 비교하여, 'query' (사용자 질문)에 대한 답변이 정확한지 평가하는 것입니다.

평가해야 할 정보는 다음과 같습니다:

1. **Evidence (정답 데이터):**
```
{evidence}
```

2. **User Query (사용자 질문):**
```
{query}
```

3. **LLM Result (LLM 생성 쿼리 실행 결과):**
```
{llm_result}
```

**평가 지침:**

1. `llm_result`가 `evidence`와 정확히 일치하는지 확인하세요.
2. 만약 결과가 다르다면, 왜 다른지, 어떤 부분이 틀렸는지 구체적으로 설명하는 피드백을 작성해야 합니다.
3. 피드백은 LLM이 더 나은 쿼리를 생성하는 데 도움이 되도록 건설적이어야 합니다.

**출력 형식:**

평가 결과를 다음 형식에 맞춰 JSON으로 작성해주세요.

```json
{{
  "correctness": "Correct" or "Incorrect",
  "feedback": "결과가 올바르다면 '결과가 정확합니다.'라고 작성합니다. 결과가 틀렸다면, 틀린 이유와 올바른 결과를 생성하기 위한 구체적인 가이드를 포함하여 상세한 피드백을 작성해주세요."
}}
```
"""

schema_info = """
Tables: [('frpm',), ('satscores',), ('schools',)]

Schema for frpm:
(0, 'CDSCode', 'TEXT', 1, None, 1)
(1, 'Academic Year', 'TEXT', 0, None, 0)
(2, 'County Code', 'TEXT', 0, None, 0)
(3, 'District Code', 'INTEGER', 0, None, 0)
(4, 'School Code', 'TEXT', 0, None, 0)
(5, 'County Name', 'TEXT', 0, None, 0)
(6, 'District Name', 'TEXT', 0, None, 0)
(7, 'School Name', 'TEXT', 0, None, 0)
(8, 'District Type', 'TEXT', 0, None, 0)
(9, 'School Type', 'TEXT', 0, None, 0)
(10, 'Educational Option Type', 'TEXT', 0, None, 0)
(11, 'NSLP Provision Status', 'TEXT', 0, None, 0)
(12, 'Charter School (Y/N)', 'INTEGER', 0, None, 0)
(13, 'Charter School Number', 'TEXT', 0, None, 0)
(14, 'Charter Funding Type', 'TEXT', 0, None, 0)
(15, 'IRC', 'INTEGER', 0, None, 0)
(16, 'Low Grade', 'TEXT', 0, None, 0)
(17, 'High Grade', 'TEXT', 0, None, 0)
(18, 'Enrollment (K-12)', 'REAL', 0, None, 0)
(19, 'Free Meal Count (K-12)', 'REAL', 0, None, 0)
(20, 'Percent (%) Eligible Free (K-12)', 'REAL', 0, None, 0)
(21, 'FRPM Count (K-12)', 'REAL', 0, None, 0)
(22, 'Percent (%) Eligible FRPM (K-12)', 'REAL', 0, None, 0)
(23, 'Enrollment (Ages 5-17)', 'REAL', 0, None, 0)
(24, 'Free Meal Count (Ages 5-17)', 'REAL', 0, None, 0)
(25, 'Percent (%) Eligible Free (Ages 5-17)', 'REAL', 0, None, 0)
(26, 'FRPM Count (Ages 5-17)', 'REAL', 0, None, 0)
(27, 'Percent (%) Eligible FRPM (Ages 5-17)', 'REAL', 0, None, 0)
(28, '2013-14 CALPADS Fall 1 Certification Status', 'INTEGER', 0, None, 0)

Schema for satscores:
(0, 'cds', 'TEXT', 1, None, 1)
(1, 'rtype', 'TEXT', 1, None, 0)
(2, 'sname', 'TEXT', 0, None, 0)
(3, 'dname', 'TEXT', 0, None, 0)
(4, 'cname', 'TEXT', 0, None, 0)
(5, 'enroll12', 'INTEGER', 1, None, 0)
(6, 'NumTstTakr', 'INTEGER', 1, None, 0)
(7, 'AvgScrRead', 'INTEGER', 0, None, 0)
(8, 'AvgScrMath', 'INTEGER', 0, None, 0)
(9, 'AvgScrWrite', 'INTEGER', 0, None, 0)
(10, 'NumGE1500', 'INTEGER', 0, None, 0)

Schema for schools:
(0, 'CDSCode', 'TEXT', 1, None, 1)
(1, 'NCESDist', 'TEXT', 0, None, 0)
(2, 'NCESSchool', 'TEXT', 0, None, 0)
(3, 'StatusType', 'TEXT', 1, None, 0)
(4, 'County', 'TEXT', 1, None, 0)
(5, 'District', 'TEXT', 1, None, 0)
(6, 'School', 'TEXT', 0, None, 0)
(7, 'Street', 'TEXT', 0, None, 0)
(8, 'StreetAbr', 'TEXT', 0, None, 0)
(9, 'City', 'TEXT', 0, None, 0)
(10, 'Zip', 'TEXT', 0, None, 0)
(11, 'State', 'TEXT', 0, None, 0)
(12, 'MailStreet', 'TEXT', 0, None, 0)
(13, 'MailStrAbr', 'TEXT', 0, None, 0)
(14, 'MailCity', 'TEXT', 0, None, 0)
(15, 'MailZip', 'TEXT', 0, None, 0)
(16, 'MailState', 'TEXT', 0, None, 0)
(17, 'Phone', 'TEXT', 0, None, 0)
(18, 'Ext', 'TEXT', 0, None, 0)
(19, 'Website', 'TEXT', 0, None, 0)
(20, 'OpenDate', 'DATE', 0, None, 0)
(21, 'ClosedDate', 'DATE', 0, None, 0)
(22, 'Charter', 'INTEGER', 0, None, 0)
(23, 'CharterNum', 'TEXT', 0, None, 0)
(24, 'FundingType', 'TEXT', 0, None, 0)
(25, 'DOC', 'TEXT', 1, None, 0)
(26, 'DOCType', 'TEXT', 1, None, 0)
(27, 'SOC', 'TEXT', 0, None, 0)
(28, 'SOCType', 'TEXT', 0, None, 0)
(29, 'EdOpsCode', 'TEXT', 0, None, 0)
(30, 'EdOpsName', 'TEXT', 0, None, 0)
(31, 'EILCode', 'TEXT', 0, None, 0)
(32, 'EILName', 'TEXT', 0, None, 0)
(33, 'GSoffered', 'TEXT', 0, None, 0)
(34, 'GSserved', 'TEXT', 0, None, 0)
(35, 'Virtual', 'TEXT', 0, None, 0)
(36, 'Magnet', 'INTEGER', 0, None, 0)
(37, 'Latitude', 'REAL', 0, None, 0)
(38, 'Longitude', 'REAL', 0, None, 0)
(39, 'AdmFName1', 'TEXT', 0, None, 0)
(40, 'AdmLName1', 'TEXT', 0, None, 0)
(41, 'AdmEmail1', 'TEXT', 0, None, 0)
(42, 'AdmFName2', 'TEXT', 0, None, 0)
(43, 'AdmLName2', 'TEXT', 0, None, 0)
(44, 'AdmEmail2', 'TEXT', 0, None, 0)
(45, 'AdmFName3', 'TEXT', 0, None, 0)
(46, 'AdmLName3', 'TEXT', 0, None, 0)
(47, 'AdmEmail3', 'TEXT', 0, None, 0)
(48, 'LastUpdate', 'DATE', 1, None, 0)
"""


SystemPrompt = f"""
저는 데이터베이스 쿼리 엔진을 위한 프롬프트 엔지니어링 전문가입니다.

{schema_info}를 참조해서 사용자의 질문을 데이터베이스 쿼리로 변환하는 것이 목표입니다.
"""

In [5]:
!uv add openai 
from openai import OpenAI
MODEL_NAME="Qwen/Qwen3-Next-80B-A3B-Instruct-FP8"
MODEL_BASE_URI="http://211.47.56.73:7972/v1"
MODEL_API_KEY="token-abc123"
client = OpenAI(api_key=MODEL_API_KEY, base_url=MODEL_BASE_URI)

Resolved 102 packages in 0.40ms
Audited 100 packages in 0.03ms


In [7]:
import json
from typing import TypedDict
from langgraph.graph import StateGraph, END

# LangGraph의 상태를 정의합니다. new_prompt를 제거하고 system_prompt가 직접 업데이트되도록 합니다.
class GraphState(TypedDict):
    system_prompt: str
    user_query: str
    evidence: str
    llm_result: str
    evaluation_result: dict

# 첫 번째 노드: SQL 쿼리를 생성합니다. (변경 없음)
def generate_sql(state):
    """
    초기 시스템 프롬프트와 사용자 질문을 기반으로 SQL 쿼리를 생성합니다.
    """
    print("--- 1. SQL 쿼리 생성 중 ---")
    system_prompt = state['system_prompt']
    user_query = state['user_query']

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_query}
        ]
    )
    sql_query = response.choices[0].message.content
    # print(f"생성된 SQL: \n{sql_query}")
    return {"llm_result": sql_query}

# 두 번째 노드: 생성된 SQL을 평가합니다. (변경 없음)
def evaluate_sql(state):
    """
    생성된 SQL 쿼리의 실행 결과를 정답(evidence)과 비교하여 정확성을 평가하고,
    JSON 형식으로 결과를 반환합니다.
    """
    print("\n--- 2. SQL 쿼리 평가 중 ---")
    user_query = state['user_query']
    evidence = state['evidence']
    llm_result = state['llm_result']

    filled_evaluation_prompt = evaluation_prompt.format(
        evidence=evidence,
        query=user_query,
        llm_result=llm_result
    )

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": filled_evaluation_prompt + "\n\n출력은 반드시 JSON 형식이어야 합니다."},
        ]
    )
    evaluation_result_str = response.choices[0].message.content
    # print(f"평가 결과 (Raw): \n{evaluation_result_str}")

    try:
        if "```json" in evaluation_result_str:
            clean_str = evaluation_result_str.split("```json\n")[1].split("\n```")[0]
        else:
            clean_str = evaluation_result_str
        evaluation_json = json.loads(clean_str)
    except (json.JSONDecodeError, IndexError) as e:
        print(f"평가 결과 JSON 파싱 오류: {e}")
        evaluation_json = {"correctness": "Incorrect", "feedback": "Evaluation JSON output parsing failed."}
        
    return {"evaluation_result": evaluation_json}

# 세 번째 노드: 시스템 프롬프트를 업데이트합니다. (반환값 변경)
def update_prompt(state):
    """
    평가 결과가 'Incorrect'일 경우, 피드백을 바탕으로 시스템 프롬프트를 개선하고,
    상태의 'system_prompt'를 직접 업데이트합니다.
    """
    print("\n--- 3. 시스템 프롬프트 업데이트 중 ---")
    evaluation_result = state['evaluation_result']
    llm_result = state['llm_result']
    current_prompt = state['system_prompt']
    
    filled_prompt_updater_prompt = prompt_updater_prompt.format(
        current_prompt=current_prompt,
        feedback_prompt=evaluation_result.get('feedback', ''),
        llm_result=llm_result
    )

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": filled_prompt_updater_prompt},
        ]
    )
    new_prompt_str = response.choices[0].message.content
    
    # print(f"업데이트된 시스템 프롬프트: \n{new_prompt_str}")
    # 'new_prompt' 대신 'system_prompt' 키를 반환하여 상태를 직접 업데이트합니다.
    return {"system_prompt": new_prompt_str}

# 조건부 엣지: 평가 결과에 따라 다음 단계를 결정합니다. (변경 없음)
def decide_to_update(state):
    """
    SQL 평가 결과를 확인하고, 'Correct'이면 워크플로우를 종료하고,
    'Incorrect'이면 프롬프트 업데이트 노드로 분기합니다.
    """
    print("\n--- 4. 다음 단계 결정 ---")
    evaluation_result = state['evaluation_result']
    if evaluation_result.get("correctness", "Incorrect").lower() == "correct":
        print("결과: 정확함. 워크플로우를 종료합니다.")
        return "end"
    else:
        print("결과: 부정확함. 프롬프트 업데이트를 시작합니다.")
        return "update_prompt"

# 그래프 구성
workflow = StateGraph(GraphState)

workflow.add_node("generate_sql", generate_sql)
workflow.add_node("evaluate_sql", evaluate_sql)
workflow.add_node("update_prompt", update_prompt)

# 엣지 연결 (루프 구조로 변경)
workflow.set_entry_point("generate_sql")
workflow.add_edge("generate_sql", "evaluate_sql")
workflow.add_conditional_edges(
    "evaluate_sql",
    decide_to_update,
    {
        "update_prompt": "update_prompt",
        "end": END
    }
)
# update_prompt 노드 실행 후, 다시 generate_sql 노드로 돌아가 루프를 형성합니다.
workflow.add_edge("update_prompt", "generate_sql")

# 그래프 컴파일
app = workflow.compile()
    

In [ ]:
!uv add pygraphviz pydot


In [ ]:
!uv add tqdm


In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
experiment_logs = []
current_system_prompt = SystemPrompt
for index, row in tqdm(hard_df.iterrows(), total=len(hard_df), desc="프롬프트 자동 튜닝 실험 진행 중"):
    user_query = row['question']
    evidence = str(row['evidence'])
    initial_state = {
        "system_prompt": current_system_prompt,
        "user_query": user_query,
        "evidence": evidence,
    }    
    print(f"\n===== [실험 {index+1}/{len(hard_df)}] 시작 =====")
    print(f"질문: {user_query[:100]}...")
    try:
        final_state = app.invoke(initial_state, config={"recursion_limit": 50})
        current_system_prompt = final_state['system_prompt']
        log_entry = {
            "index": index,
            "question": user_query,
            "evidence": evidence,
            "initial_system_prompt": initial_state['system_prompt'],
            "final_system_prompt": current_system_prompt,
            "final_sql": final_state.get('llm_result', 'N/A'),
            "evaluation_correctness": final_state.get('evaluation_result', {}).get('correctness', 'N/A'),
            "evaluation_feedback": final_state.get('evaluation_result', {}).get('feedback', 'N/A')
        }
        experiment_logs.append(log_entry)
        
        print(f"===== [실험 {index+1}/{len(hard_df)}] 종료: {'성공' if log_entry['evaluation_correctness'].lower() == 'correct' else '개선 필요'} =====")

    except Exception as e:
        print(f"!!!!! [실험 {index+1}/{len(hard_df)}] 오류 발생 !!!!!")
        print(str(e))
        # 오류가 발생한 경우에도 로그를 남깁니다.
        log_entry = {
            "index": index,
            "question": user_query,
            "evidence": evidence,
            "initial_system_prompt": current_system_prompt,
            "final_system_prompt": "ERROR",
            "final_sql": "ERROR",
            "evaluation_correctness": "ERROR",
            "evaluation_feedback": str(e)
        }
        experiment_logs.append(log_entry)
        continue # 오류 발생 시 다음 데이터로 넘어갑니다.

# 최종적으로 개선된 시스템 프롬프트를 출력합니다.
print("\n\n==================== 최종 시스템 프롬프트 ====================")
print(current_system_prompt)

# 로그 리스트를 데이터프레임으로 변환하고 CSV 파일로 저장합니다.
log_df = pd.DataFrame(experiment_logs)
log_df.to_csv("experiment_log.csv", index=False, encoding='utf-8-sig')

print("\n\n실험 결과가 'experiment_log.csv' 파일로 저장되었습니다.")
log_df.head()


프롬프트 자동 튜닝 실험 진행 중:   0%|          | 0/231 [00:00<?, ?it/s]


===== [실험 1/231] 시작 =====
질문: For the school with the highest free meal rate in Alameda County, what are its characteristics inclu...
--- 1. SQL 쿼리 생성 중 ---

--- 2. SQL 쿼리 평가 중 ---

--- 4. 다음 단계 결정 ---
결과: 부정확함. 프롬프트 업데이트를 시작합니다.

--- 3. 시스템 프롬프트 업데이트 중 ---
--- 1. SQL 쿼리 생성 중 ---

--- 2. SQL 쿼리 평가 중 ---

--- 4. 다음 단계 결정 ---
결과: 부정확함. 프롬프트 업데이트를 시작합니다.

--- 3. 시스템 프롬프트 업데이트 중 ---
--- 1. SQL 쿼리 생성 중 ---

--- 2. SQL 쿼리 평가 중 ---

--- 4. 다음 단계 결정 ---
결과: 정확함. 워크플로우를 종료합니다.
===== [실험 1/231] 종료: 성공 =====

===== [실험 3/231] 시작 =====
질문: For charter schools in Fresno County Office of Education, provide their location details, enrollment...
--- 1. SQL 쿼리 생성 중 ---

--- 2. SQL 쿼리 평가 중 ---

--- 4. 다음 단계 결정 ---
결과: 부정확함. 프롬프트 업데이트를 시작합니다.

--- 3. 시스템 프롬프트 업데이트 중 ---
--- 1. SQL 쿼리 생성 중 ---

--- 2. SQL 쿼리 평가 중 ---

--- 4. 다음 단계 결정 ---
결과: 정확함. 워크플로우를 종료합니다.
===== [실험 3/231] 종료: 성공 =====

===== [실험 4/231] 시작 =====
질문: For the non-charter school with more than 100 students that has the highest number of

In [ ]:
# tqdm의 노트북용 위젯을 사용하기 위해 ipywidgets를 설치합니다.
# 이 셀을 실행한 후에는 반드시 [Kernel] -> [Restart Kernel]을 선택하여 커널을 다시 시작해주세요.
!uv add ipywidgets


In [ ]:
hard_df

In [ ]:
from IPython.display import Image, display

try:
    # app 객체에서 그래프를 가져와 PNG 이미지로 변환 후 출력합니다.
    png_image = app.get_graph().draw_mermaid_png()
    display(Image(png_image))
except Exception as e:
    print(f"그래프를 시각화하는 중 오류가 발생했습니다: {e}")
    print("\nGraphviz 라이브러리가 시스템에 설치되어 있는지 확인해주세요.")
    print("Ubuntu/Debian: sudo apt-get install graphviz")
    print("macOS (Homebrew): brew install graphviz")


In [ ]:
hard_df.head()

In [ ]:
for index, row in hard_df.iterrows():
    user_query = row['question']
    evidence = row['evidence']
    gt_sql_query = row['SQL']
    print(user_query)
    print(evidence)
    print(gt_sql_query)

In [ ]:
llm_result = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": SystemPrompt},
        {"role": "user", "content": user_query}
    ]
)
print(llm_result.choices[0].message.content)
evaluation_result = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": evaluation_prompt + "한글로 말해줘"},
        {"role": "user", "content": llm_result.choices[0].message.content},
        {"role": "user", "content": evidence},
        {"role": "user", "content": gt_sql_query}
    ]
)
print(evaluation_result.choices[0].message.content)
prompt_updater_result = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": prompt_updater_prompt},
        {"role": "user", "content": "evaluation_result: " + evaluation_result.choices[0].message.content},
        {"role": "user", "content": "llm_result: " + llm_result.choices[0].message.content},
        {"role": "user", "content": "current_prompt: " + SystemPrompt}
    ]
)
print(prompt_updater_result.choices[0].message.content)
